## 00 - Preparamos el entorno cargando las bibliotecas necesarias
En caso de que dichas bibliotecas no estén instaladas, es necesario ejecutar el siguiente bloque de código
```
! pip install --upgrade requests numpy pandas
```

In [1]:
import requests
import numpy as np
import json
import pandas as pd

## 01 - Preparación de la solicitud
Cuando interactuar con el servidor [www.transparenciapresupuestaria.gob.mx](https://www.transparenciapresupuestaria.gob.mx), dicho servidor espera información por parte nuestra.

En el navegador, el frontend se encarga de preparar el request mediante la interacción con el usuario

<img src="transparenciapresupuestaria.png" width="500px"/>

El servidor remoto usa la api albergada en [https://nptp.hacienda.gob.mx/programas/mapa](https://nptp.hacienda.gob.mx/programas/mapa).

Qué pasa si haces click en el link anterior?

### 01.1 - Un diccionario simple
En la siguiente celda, podrás construir un diccionario en el que se almacenan los distintos parámetros que le podemos pasar a la API de Transparencia Presupuestaria

In [2]:
url_data = {"id_entidad_federativa":"1",
            "id_ramo":"",
            "desc_ppi":"",
            "id_clave_cartera":"",
            "page":"1",
            "pageSize":"20"}

### 01.1 - Limpieza de datos

Para un humano es sencillo notar si algo está vacío, sencillamente lo omites y listo.

Para una API esto no siempre es obvio, por lo que antes de lanzar nuestra solicitud, debemos limpiar el diccionario, para no pasarle parámetros vacíos a la solicitud

In [3]:
remove_keys = []
for key in url_data.keys():
    if (url_data[key] == ""):
        remove_keys.append(key)
for key in remove_keys:
    del url_data[key]

In [4]:
print(url_data)

{'id_entidad_federativa': '1', 'page': '1', 'pageSize': '20'}


## 02 - Solicitud usando requests

Ya que limpiamos el set de parámetros que le vamos a pasar a la URL de la API, podemos lanzar la solicitud

No obstante, si el set de parámetros está vacío, mejor es no pasarle nada a la API y usar los defaults

In [5]:
if(len(url_data.keys())==0):
    url_request = requests.get('https://nptp.hacienda.gob.mx/programas/mapa')
else:
    url_request = requests.get('https://nptp.hacienda.gob.mx/programas/mapa',params=url_data)

### 02.1 Safety checks

Si todo salió bien con el código anterior, podemos revisar algunos elementos de nuestra solicitud

Al ejecutar el código en la siguiente celda podemos ver el URL de la API.

Puedes hacer click en el link y revisar el output en el navegador

>**Feeling daring?**

>Tambien puedes usar [curl](https://curl.se/) y [JQ](https://jqlang.github.io/jq/) para llamar a la API y obtener resultados de consulta en tu terminal

>La regla no escrita en cómputo es que siempre hay más de una manera de hacer las cosas

In [6]:
print(url_request.url)

https://nptp.hacienda.gob.mx/programas/mapa?id_entidad_federativa=1&page=1&pageSize=20


### 02.2 Status code

Si bien realizamos la solicitud exitosamente, no significa que el servidor haya aceptado dicha solicitud

Podemos checar el status code como una instancia de requests

Y en una sola línea, podemos compararlo con códigos permisibles

In [7]:
if (url_request.status_code == requests.codes.ok):
    print("Solicitud efectuada exitosamente")
else:
    print("Error en la solicitud"+str(url_request.status_code))

Solicitud efectuada exitosamente


## 03 - Data retrieval

Si nuestra solicitud se llevó a cabo exitosamente, el servidor nos entregará la información en formato [JSON](https://en.wikipedia.org/wiki/JSON)

>***Es sumamente importante revisar la API de cada sitio web, aún hay sitios que entregan XML, texto plano o incluso HTML***

>Los parseadores que usemos para obtener información de cada sitio deberán ajustarse al formato que el servidor entreguen

In [8]:
if (url_request.status_code == requests.codes.ok):
    retrieved_data = url_request.json()

### 03.1 Resumen de nuestros datos

Antes de procesar los datos es indispensable tener una idea de:

- Qué tamaño tienen
- Cómo están estructurados
- Qué tipo de datos están almacenados en cada *campo*

#### 03.1.1 Tamaño
JSON es un formato que puede ser tan superficial o tan profundo cómo queramos, con el siguiente código podemos ver cuantos elementos nos mandó el servidor a nivel superficial

In [9]:
print(len(retrieved_data))

5


#### 03.1.2 Estructura

En las siguientes celdas podemos obtener información relevante acerca de la estructura de la información obtenida del servidor remoto

In [10]:
type(retrieved_data)

dict

In [11]:
retrieved_keys = retrieved_data.keys()
print(retrieved_keys)

dict_keys(['total', 'results', 'page', 'pageSize', 'pages'])


#### 03.1.3 Tipo de datos

La información obtenida está organizada en 5 elementos superficiales, sin embargo, uno de ellos tiene más de lo que aparenta

In [12]:
for key in retrieved_keys:
    print(key + "\t" +str(type(retrieved_data[key])))

total	<class 'int'>
results	<class 'list'>
page	<class 'int'>
pageSize	<class 'int'>
pages	<class 'float'>


#### 03.1.4 Estructura, again

El elemento "results" dentro de nuestro objeto de python es de tipo `list`

Ésto nos indica que podemos iterar sobre dicha lista y accesar los valores presentes en ella

In [13]:
print(len(retrieved_data["results"]))

20


In [14]:
print(type(retrieved_data["results"][0]))

<class 'dict'>


In [15]:
print(retrieved_data["results"][0].keys())

dict_keys(['id_clave_cartera', 'desc_ppi', 'id_ramo', 'id_entidad_federativa', 'latitud', 'longitud', 'monto_inversion_total', 'ciclo', 'fase', 'desc_ramo', 'desc_entidad_federativa', 'tipo_proyecto'])


In [16]:
results_keys = retrieved_data["results"][0].keys()
for key in results_keys:
    print(key + "\t" +str(type(retrieved_data["results"][0][key])))

id_clave_cartera	<class 'str'>
desc_ppi	<class 'str'>
id_ramo	<class 'int'>
id_entidad_federativa	<class 'str'>
latitud	<class 'float'>
longitud	<class 'float'>
monto_inversion_total	<class 'int'>
ciclo	<class 'int'>
fase	<class 'str'>
desc_ramo	<class 'str'>
desc_entidad_federativa	<class 'str'>
tipo_proyecto	<class 'str'>


## 04 - Obteniendo información básica

### 04.1 Elementos superficiales

Los elementos superficiales del diccionario `retrieved_data` los podemos accesar directamente a través del nombre de su llave

In [17]:
entries   = retrieved_data["total"]
page      = retrieved_data["page"]
pages     = retrieved_data["pages"]
page_size = retrieved_data["pageSize"]

### 04.2 Elementos anidados

Los elementos anidados, como `results`, tenemos que sub indizarlos, es decir, primero llamamos la llave results y luego el índice de cada sub elemento

Para accesar al primer elemento llamamos al índice `[0]`

In [18]:
retrieved_data["results"][0]

{'id_clave_cartera': '11096210010',
 'desc_ppi': 'Tercer Anillo, Primera Etapa.',
 'id_ramo': 9,
 'id_entidad_federativa': '01',
 'latitud': 21.9523,
 'longitud': -102.3529,
 'monto_inversion_total': 906008771,
 'ciclo': 2024,
 'fase': 'Calendario Fiscal Concluido / Operación',
 'desc_ramo': 'Comunicaciones y Transportes',
 'desc_entidad_federativa': 'AGUASCALIENTES',
 'tipo_proyecto': 'Proyecto de Inversión de Infraestructura Económica'}

In [19]:
type(retrieved_data["results"][0])

dict

## 05 - Nuestro mejor amigo el diccionario

Los desarrolladores detrás de las APIs usualmente muestran los datos de forma bien estructurada.

Podemos tomar ventaja de ello y convertir todo el elemento `results` en un dataframe usando pandas

In [20]:
df = pd.DataFrame.from_dict(retrieved_data["results"])

In [21]:
df

,id_clave_cartera,desc_ppi,id_ramo,id_entidad_federativa,latitud,longitud,monto_inversion_total,ciclo,fase,desc_ramo,desc_entidad_federativa,tipo_proyecto
0,11096210010,"Tercer Anillo, Primera Etapa.",9,01,21.952300,-102.352900,906008771,2024,Calendario Fiscal Concluido / Operación,Comunicaciones y Transportes,AGUASCALIENTES,Proyecto de Inversión de Infraestructura Econó...
1,1807HXA0019,Mantenimiento y rehabilitación de la UHM Calvi...,7,01,0.000000,0.000000,17813128,2024,Calendario Fiscal Concluido / Operación,Defensa Nacional,AGUASCALIENTES,Programa de Inversión de Mantenimiento
2,1950GYR0005,Premio IMSS a la Competitividad. Programa de s...,50,01,21.901375,-102.291674,478887,2024,Calendario Fiscal Concluido / Operación,Instituto Mexicano del Seguro Social,AGUASCALIENTES,Programa de Inversión de Adquisiciones
3,2006HJO0005,Construcción y Adquisición de Mobiliario y Equ...,6,01;02;03;04;06;07;10;11;12;13;14;15;16;17;18;1...,0.000000,0.000000,643148483,2024,Vigente,Hacienda y Crédito Público,AGUASCALIENTES;BAJA CALIFORNIA;BAJA CALIFORNIA...,Proyecto de Inversión de Infraestructura Guber...
4,2006HJO0006,Inversión para la construcción de 90 sucursale...,6,01;02;03;05;06;08;10;11;13;14;16;17;18;19;21;2...,0.000000,0.000000,483115190,2024,Vigente,Hacienda y Crédito Público,AGUASCALIENTES;BAJA CALIFORNIA;BAJA CALIFORNIA...,Proyecto de Inversión de Infraestructura Guber...
5,2006HJO0009,Inversión 89 nuevas sucursales Banco del Biene...,6,01;03;04;05;07;08;09;10;12;13;15;16;17;18;19;20,0.000000,0.000000,477747243,2024,Vigente,Hacienda y Crédito Público,AGUASCALIENTES;BAJA CALIFORNIA SUR;CAMPECHE;CO...,Proyecto de Inversión de Infraestructura Guber...
6,2050GYR0008,Estudios de Preinversión para el desarrollo de...,50,01;03;14;21;26,23.078357,-109.714003,55378654,2024,En Proceso de Modificación,Instituto Mexicano del Seguro Social,AGUASCALIENTES;BAJA CALIFORNIA SUR;JALISCO;PUE...,Programa de Estudios de Preinversión
7,2050GYR0032,Premio IMSS a la Competitividad. Programa de s...,50,01,21.901380,-102.291670,485345,2024,Calendario Fiscal Concluido / Operación,Instituto Mexicano del Seguro Social,AGUASCALIENTES,Programa de Inversión de Adquisiciones
8,2106HJO0002,Inversión 85 nuevas sucursales del Banco del B...,6,01;02;05;06;07;08;09;10;11;16;19;22;24;25;26,0.000000,0.000000,462328905,2024,Vigente,Hacienda y Crédito Público,AGUASCALIENTES;BAJA CALIFORNIA;COAHUILA DE ZAR...,Proyecto de Inversión de Infraestructura Guber...
9,2106HJO0007,Sustitución y adquisición adicional de equipo ...,6,01;02;03;04;05;06;07;08;09;10;11;12;13;14;15;1...,0.000000,0.000000,270826417,2024,Vigente,Hacienda y Crédito Público,AGUASCALIENTES;BAJA CALIFORNIA;BAJA CALIFORNIA...,Programa de Inversión de Adquisiciones


### 05.1 Ahorrandonos unos clicks

Ya que tenemos nuestro dataframe, podemos exportarlo a algo más legible y transportable

In [22]:
df.to_csv("resultados.tsv",sep="\t",index=False)

## 06 - Entrando al agujero de conejo

En el sitio de transparencia presupuestaria puedes consultar más información acerca de cada proyecto

El sitio te lleva a la página [https://www.transparenciapresupuestaria.gob.mx/ficha_opa](https://www.transparenciapresupuestaria.gob.mx/ficha_opa)

Sin embargo, en dicha página no puedes hacer consultas si no es desde la página anterior (Obra Pública Abierta)

Si analizamos nuestro entorno de desarrollo en el navegador, veremos que se hace una llamada a una segunda API

Dicha API está albergada en [https://nptp.hacienda.gob.mx/programas/detalle](https://nptp.hacienda.gob.mx/programas/detalle

Y espera como único parámetro `id_clave_cartera`

### 06.1 Preparando el acceso por lote

Ya sea desde nuestro dataframe o desde nuestro objeto JSON, podemos obtener una lista de los valores de `id_clave_cartera`

Con dicha lista podemos interrogar la segunda API para obtener información más detallada de cada proyecto

>Spoiler, encontraremos más información de la que se muestra en el sitio web!

Primero construimos una lista y la llenamos con los valores de `id_clave_cartera`

In [23]:
id_clave_cartera_list = []
for entry in range(page_size):
    id_clave_cartera_list.append(retrieved_data["results"][entry]["id_clave_cartera"])

In [24]:
id_clave_cartera_list

['11096210010',
 '1807HXA0019',
 '1950GYR0005',
 '2006HJO0005',
 '2006HJO0006',
 '2006HJO0009',
 '2050GYR0008',
 '2050GYR0032',
 '2106HJO0002',
 '2106HJO0007',
 '2106HJO0015',
 '2106HJO0026',
 '2106HJO0030',
 '21096210001',
 '2116B000056',
 '2150GYR0021',
 '2150GYR0036',
 '2150GYR0059',
 '2206E000004',
 '2236H000007']

### 06.2 Una prueba rápida

Antes de lanzar toda la lista de ids, probemos con solamente un elemento.

Haremos lo mismo que en la primera parte, prepararemos un diccionario que contiene los parámetros que le pasaremos a la API

In [25]:
ext_data = {"id_clave_cartera" : "11096210010"}

#### 06.2.1 Igualmente haremos una limpieza de datos

In [26]:
remove_keys = []
for key in ext_data.keys():
    if (ext_data[key] == ""):
        remove_keys.append(key)
for key in remove_keys:
    del ext_data[key]
if(len(ext_data.keys())!=0):
    ext_request = requests.get("https://nptp.hacienda.gob.mx/programas/detalle",params=ext_data)

In [27]:
print(ext_request.url)

https://nptp.hacienda.gob.mx/programas/detalle?id_clave_cartera=11096210010


#### 06.2.2 Status code

In [28]:
if (ext_request.status_code == requests.codes.ok):
    print("Solicitud efectuada exitosamente")
else:
    print("Error en la solicitud"+str(ext_request.status_code))

Solicitud efectuada exitosamente


In [29]:
if (ext_request.status_code == requests.codes.ok):
    retrieved_details = ext_request.json()

#### 06.2.3 Exploración de los datos

En este punto ya es más sencillo ir directo sobre los datos

Vamos a analizar el objeto recibido y pensemos en formas de transformar la información recibida en formatos más manejables

Pensemos tambien en cuales serían los siguientes pasos para la automatización de extracción de información, tanto de la primera como de la segunda API

In [30]:
retrieved_details

[{'id_ppi': 30283,
  'id_clave_cartera': '11096210010',
  'desc_ppi': 'Tercer Anillo, Primera Etapa.',
  'fase': 'Calendario Fiscal Concluido / Operación',
  'id_tipo_ppi': 1,
  'desc_tipo_ppi': 'Proyecto de Inversión de Infraestructura Económica',
  'detalle_ppi': 'El Tercer Anillo Periférico en su parte Poniente  constará de un tramo nuevo que alojará 2 calzadas, una para cada sentido de circulación con 3 carriles de 3.5 m de ancho a lo largo de 12.85 km, además de 3 PSV, para dar continuidad al flujo vehicular.',
  'latitud': 21.9523,
  'longitud': -102.3529,
  'nombre_admin': 'Oscar Raúl',
  'apellido_paterno_admin': 'Callejo',
  'apellido_materno_admin': 'Silva',
  'cargo_admin': 'Director General de Carreteras',
  'mail_admin': 'ocallejo@sct.gob.mx',
  'telefono_admin': None,
  'beneficios_esperados': 'Dar continuidad al flujo vehicular por el Tercer Anillo de oriente a poniente y note a sur, aumentar las velocidades de operación, reducir los tiempos de recorrido, reducir los cos

In [31]:
for key in retrieved_details[0].keys():
    print(key+"\t"+str(type(retrieved_details[0][key])))

id_ppi	<class 'int'>
id_clave_cartera	<class 'str'>
desc_ppi	<class 'str'>
fase	<class 'str'>
id_tipo_ppi	<class 'int'>
desc_tipo_ppi	<class 'str'>
detalle_ppi	<class 'str'>
latitud	<class 'float'>
longitud	<class 'float'>
nombre_admin	<class 'str'>
apellido_paterno_admin	<class 'str'>
apellido_materno_admin	<class 'str'>
cargo_admin	<class 'str'>
mail_admin	<class 'str'>
telefono_admin	<class 'NoneType'>
beneficios_esperados	<class 'str'>
meta_unidad_medida	<class 'str'>
monto_inversion_total	<class 'int'>
anios_operacion_total	<class 'int'>
monto_operacion_total	<class 'int'>
otros_gastos	<class 'int'>
monto_total	<class 'int'>
fecha_inicio_calendario_fiscal	<class 'str'>
fecha_fin_calendario_fiscal	<class 'str'>
anio_inversion	<class 'str'>
ciclo	<class 'int'>
total_contratos_ejecucion	<class 'int'>
url_contratos	<class 'str'>
ent_federativa	<class 'list'>
pp	<class 'list'>
ur	<class 'list'>
version	<class 'str'>


In [32]:
useful_keys = []